# Install KenLM Library

In [1]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mv kenlm kenlm_tool
!mkdir kenlm_tool/build

--2025-02-04 05:29:37--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 129.80.89.152, 2603:c020:4009:8710:ca:11:17:0
Connecting to kheafield.com (kheafield.com)|129.80.89.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  2.90MB/s    in 0.2s    

2025-02-04 05:29:37 (2.90 MB/s) - written to stdout [491888/491888]



In [2]:
%cd kenlm_tool/build
!cmake ..
!make -j2

/content/kenlm_tool/build
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
CMake Warning (dev) at CMakeLists.txt:85 (find_packa

# Download Data

In [3]:
!gdown 1g6zRU45wZqZjLyOPt_4BNft8DXUuxGgQ

Downloading...
From: https://drive.google.com/uc?id=1g6zRU45wZqZjLyOPt_4BNft8DXUuxGgQ
To: /content/kenlm_tool/build/gigaword3_nyt_eng_200001.tar.gz
100% 17.6M/17.6M [00:00<00:00, 116MB/s] 


In [4]:
!tar -xf gigaword3_nyt_eng_200001.tar.gz

In [5]:
ls

bin/            CMakeFiles/          gigaword3_nyt_eng_200001.tar.gz  lib/  Makefile        util/
CMakeCache.txt  cmake_install.cmake  kenlmConfig.cmake                lm/   nyt_eng_200001


# Install KenLM in Python

In [6]:
!pip install https://github.com/kpu/kenlm/archive/master.zip


     | 553.6 kB 3.6 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp311-cp311-linux_x86_64.whl size=3186615 sha256=1f2af59002983b94594cd8b871351b68ecef55bc436c2afaaefcad9b45f4059e
  Stored in directory: /tmp/pip-ephem-wheel-cache-51majmmd/wheels/4e/ca/6a/e5da175b1396483f6f410cdb4cfe8bc8fa5e12088e91d60413
Successfully built kenlm


# Train LM with KenLM

In [7]:
!bin/lmplz -o 5 --text nyt_eng_200001 --arpa fivegram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm_tool/build/nyt_eng_200001
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 9246188 types 145846
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1750152 2:1062028736 3:1991303936 4:3186086144 5:4646375936
Statistics:
1 145846 D1=0.631447 D2=1.00426 D3+=1.35926
2 1687095 D1=0.739818 D2=1.09499 D3+=1.40095
3 4295889 D1=0.837724 D2=1.20195 D3+=1.3587
4 5632714 D1=0.909669 D2=1.36184 D3+=1.21414
5 5673278 D1=0.767442 D2=1.58786 D3+=0.567867
Memory estimate for binary LM:
type     MB
probing 366 assuming -p 1.5
probing 433 assuming -r models -p 1.5
trie    180 without quantization
trie     99 assuming -q 8 -b 8 quantization 
trie    158 assuming -a 22 array pointer compression
trie     78 assuming -a 22 -q 8 -b 8 array pointer compression and qu

In [8]:
ls

bin/            cmake_install.cmake              kenlmConfig.cmake  Makefile
CMakeCache.txt  fivegram.arpa                    lib/               nyt_eng_200001
CMakeFiles/     gigaword3_nyt_eng_200001.tar.gz  lm/                util/


In [9]:
import kenlm
model = kenlm.Model('fivegram.arpa')

In [10]:
import math
import pandas as pd
def print_score(model, s):
  tokens = s.split(' ')
  log_score = 0.0
  rows = []
  for i, (logprob, length, oov) in enumerate(model.full_scores(s)):
    if i < len(tokens):
      row = {'token': tokens[i], 'probability': math.exp(logprob), 'Is OOV?': oov}
    else:
      row = {'token': 'END', 'probability': math.exp(logprob), 'Is OOV?': oov}
    rows.append(row)
    log_score += logprob
  print ('Log probability = ', log_score)
  return pd.DataFrame(rows)

In [11]:
print_score(model, 'I look forward to meeting you')

Log probability =  -13.25155920535326


,token,probability,Is OOV?
0,I,0.063775,False
1,look,0.075406,False
2,forward,0.258621,False
3,to,0.914503,False
4,meeting,0.091944,False
5,you,0.034129,False
6,END,0.492458,False


In [12]:
print_score(model, 'I look forward to meet you')

Log probability =  -13.628677017986774


,token,probability,Is OOV?
0,I,0.063775,False
1,look,0.075406,False
2,forward,0.258621,False
3,to,0.914503,False
4,meet,0.045561,False
5,you,0.106006,False
6,END,0.219436,False


In [13]:
print_score(model, 'The president ceases trading to China yesterday')

Log probability =  -27.380770802497864


,token,probability,Is OOV?
0,The,0.185861,False
1,president,0.095317,False
2,ceases,0.001185,False
3,trading,0.014776,False
4,to,0.082064,False
5,China,0.028103,False
6,yesterday,0.007419,False
7,END,0.242007,False


In [14]:
print_score(model, 'The president ceased trading with China yesterday')

Log probability =  -25.994827032089233


,token,probability,Is OOV?
0,The,0.185861,False
1,president,0.095317,False
2,ceased,0.002069,False
3,trading,0.012584,False
4,with,0.145422,False
5,China,0.037907,False
6,yesterday,0.008345,False
7,END,0.242007,False


In [15]:
print_score(model, 'This essay will discuss about the current issues of climate change')

Log probability =  -35.94411849975586


,token,probability,Is OOV?
0,This,0.061282,False
1,essay,0.003959,False
2,will,0.051419,False
3,discuss,0.021940,False
4,about,0.043290,False
5,the,0.348546,False
6,current,0.044336,False
7,issues,0.017161,False
8,of,0.280364,False
9,climate,0.007410,False


In [16]:
print_score(model, 'This essay will discuss the current issues of climate change')

Log probability =  -32.93786382675171


,token,probability,Is OOV?
0,This,0.061282,False
1,essay,0.003959,False
2,will,0.051419,False
3,discuss,0.021940,False
4,the,0.313912,False
5,current,0.041336,False
6,issues,0.017882,False
7,of,0.280364,False
8,climate,0.007410,False
9,change,0.184421,False
